In [1]:
import pandas as pd
import csv
import os,sys,json
from datetime import datetime, timedelta, date
import matplotlib
import matplotlib.pyplot as plt
#import matplotlib.pylab as plt
import re
#from pylab import *
import tqdm
import numpy as np
%matplotlib inline
font = {'family' : 'Noto Sans CJK JP'}
matplotlib.rc('font', **font)

In [2]:
%%html

<style>
    .jp-OutputArea-child {
        display: inline-block;
    }
</style>

In [3]:
def display_full(df):
    max_rows = pd.get_option('display.max_rows')
    pd.set_option('display.max_rows', None)
    display(df)
    pd.set_option('display.max_rows', max_rows)

In [4]:
!mkdir -p data/google/forecasts
!wget https://storage.googleapis.com/covid-external/forecast_JAPAN_PREFECTURE_28.csv -O data/google/forecasts/forecast_JAPAN_PREFECTURE_28-`date +%Y%m%d`.csv

--2021-04-27 20:47:08--  https://storage.googleapis.com/covid-external/forecast_JAPAN_PREFECTURE_28.csv
storage.googleapis.com (storage.googleapis.com) をDNSに問いあわせています... 2404:6800:4004:806::2010, 2404:6800:4004:808::2010, 2404:6800:4004:809::2010, ...
storage.googleapis.com (storage.googleapis.com)|2404:6800:4004:806::2010|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 459349 (449K) [application/octet-stream]
`data/google/forecasts/forecast_JAPAN_PREFECTURE_28-20210427.csv' に保存中

data/google/forecas 100%[===================>] 448.58K  --.-KB/s    in 0.06s   

2021-04-27 20:47:08 (7.77 MB/s) - `data/google/forecasts/forecast_JAPAN_PREFECTURE_28-20210427.csv' へ保存完了 [459349/459349]



In [5]:
target_date = datetime.datetime.now().strftime("%Y%m%d")
df = pd.read_csv("data/google/forecasts/forecast_JAPAN_PREFECTURE_28-%s.csv"%target_date)
df.set_index("target_prediction_date").sort_index()

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
df.columns

In [ ]:
#total_df = df.loc[~df["new_confirmed"].isnull()].groupby("target_prediction_date").sum()
total_df = df.groupby("target_prediction_date").sum()
total_df

In [ ]:
!mkdir -p google-covid19-forecast/jp
ax=total_df.replace(0, np.nan).plot(title="Japan COVID19 forecast", y=["new_confirmed","new_confirmed_ground_truth"])
display(ax.figure)
plt.savefig("google-covid19-forecast/jp/total.png")
plt.close()

In [ ]:
if False:
    for p, ddf in df.groupby("prefecture_name"):
        ax=ddf.set_index("target_prediction_date").sort_index().plot(title=p, y=["new_confirmed","new_confirmed_ground_truth"])
        display(ax.figure)
        plt.savefig("google-covid19-forecast/jp/%s.png"%p)
        plt.close()

In [ ]:
from functools import reduce

def read_one_week():
    today = datetime.datetime.now()
    stable_cols = ("target_prediction_date", "prefecture_name", "new_confirmed_ground_truth")
    def data_for_date(target_date):
        filename = "data/google/forecasts/forecast_JAPAN_PREFECTURE_28-%s.csv"%target_date
        if not os.path.exists(filename):
            return None
        ddf = pd.read_csv(filename)
        result = ddf[[
            "target_prediction_date", "prefecture_name", 
            "new_confirmed_ground_truth", "new_confirmed"]]
        result["target_prediction_date"] = pd.to_datetime(result["target_prediction_date"])
        columns={ 
            c: "%s_%s"%(c, target_date) if c not in stable_cols else c for c in result.columns 
        }
        result = result.rename(columns=columns)
        return result
    
    dates = [(today - datetime.timedelta(days=i)).strftime("%Y%m%d") for i in range(6, -1, -1)]
    data  = [data_for_date(d) for d in dates]
    truth = [d[1].set_index(['target_prediction_date', 'prefecture_name'])["new_confirmed_ground_truth"].rename(d[0]) for d in zip(dates, data) if not d[1] is None]
    truth = reduce(lambda l, r: pd.merge(l, r, on=['target_prediction_date', 'prefecture_name'], how='outer'), truth)
    if len(truth.shape) > 1 and truth.shape[-1] > 1:
        truth = truth.max(axis=1)
    truth = truth.reset_index().set_index("target_prediction_date").sort_index()
    truth_avg = truth.groupby("prefecture_name").rolling(7, center=True, min_periods=1).mean()
    truth_avg = truth_avg.reset_index().set_index(["target_prediction_date", "prefecture_name"]).sort_index()
    truth = truth.reset_index().set_index(["target_prediction_date", "prefecture_name"]).sort_index()
    truth = pd.concat([truth,truth_avg], axis=1, ignore_index=True)
    truth = truth.rename(columns={0: "new_confirmed", 1: "new_confirmed(avg)"})
    data = [d.drop(columns="new_confirmed_ground_truth") for d in data if not d is None]
    data = [truth] + data
    return reduce(lambda l, r: pd.merge(l, r, on=['target_prediction_date', 'prefecture_name'], how='outer'), data)
weekly_comparison = read_one_week()
display(weekly_comparison)

In [ ]:
!mkdir -p google-covid19-forecast/jp

ax = weekly_comparison.groupby("target_prediction_date").sum().replace(0, np.nan).plot(title="Japan COVID19 forecast")
display(ax.figure)
plt.savefig("google-covid19-forecast/jp/JPN-weekly.png")
plt.close()



for p, ddf in weekly_comparison.groupby("prefecture_name"):
    ax=ddf.set_index("target_prediction_date").sort_index().plot(title=p)
    display(ax.figure)
    plt.savefig("google-covid19-forecast/jp/%s-weekly.png"%p)
    plt.close()

In [ ]:
!zip google-covid19-forecast.zip google-covid19-forecast/*